In [1]:
import numpy as np
import cv2
import os
import math
import matplotlib.pyplot as plt

In [17]:
def face_detection(image):
    #cv2.imshow('2',image)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_frontalface_default.xml')

    face = haar_classifier.detectMultiScale(image_gray, scaleFactor=1.3, minNeighbors=7)
#    mouth_cascade = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_mcs_mouth.xml') 
#    nose_cascade = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_mcs_nose.xml') 
#    eye_cascade = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_eye.xml')
    
    if len(face) != 0:
        (x,y,w,h) = face[0]
        
        dim = (100,100)
        img = image_gray[y:y+w, x:x+h]
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        return resized, face[0]
    else:
        return -1



'''
img = cv2.imread('test/10.jpg')
img,face,eye1,eye2,mouth,mouth = face_detection(img)
img = cv2.rectangle(img, (nose[0],nose[1]), (nose[0]+nose[2],nose[1]+nose[3]), (255,0,0), 2)

cv2.imshow('Test Image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


eye_cascade = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_eye.xml') 
eye = eye_cascade.detectMultiScale(img)

mouth_cascade = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_mcs_mouth.xml') 
mouth = mouth_cascade.detectMultiScale(img, 1.7, 11)
mouth = mouth[0]

nose_cascade = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_mcs_nose.xml') 
nose = nose_cascade.detectMultiScale(img, 1.3, 5)
nose = nose[0]



(x0,y0,w0,h0) = eye[0]
(x1,y1,w1,h1) = eye[1]
(x2,y2,w2,h2) = mouth
(x3,y3,w3,h3) = nose




img = cv2.rectangle(img, (x0,y0), (x0+w0,y0+h0), (255,0,0), 2)
img = cv2.rectangle(img, (x1,y1), (x1+w1,y1+h1), (255,0,0), 2)
img = cv2.rectangle(img, (x2,y2), (x2+w2,y2+h2), (255,0,0), 2)
img = cv2.rectangle(img, (x3,y3), (x3+w3,y3+h3), (255,0,0), 2)

cv2.imshow('Test Image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

"\nimg = cv2.imread('test/10.jpg')\nimg,face,eye1,eye2,mouth,mouth = face_detection(img)\nimg = cv2.rectangle(img, (nose[0],nose[1]), (nose[0]+nose[2],nose[1]+nose[3]), (255,0,0), 2)\n\ncv2.imshow('Test Image',img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n\n\neye_cascade = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_eye.xml') \neye = eye_cascade.detectMultiScale(img)\n\nmouth_cascade = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_mcs_mouth.xml') \nmouth = mouth_cascade.detectMultiScale(img, 1.7, 11)\nmouth = mouth[0]\n\nnose_cascade = cv2.CascadeClassifier('opencv/data/haarcascades/haarcascade_mcs_nose.xml') \nnose = nose_cascade.detectMultiScale(img, 1.3, 5)\nnose = nose[0]\n\n\n\n(x0,y0,w0,h0) = eye[0]\n(x1,y1,w1,h1) = eye[1]\n(x2,y2,w2,h2) = mouth\n(x3,y3,w3,h3) = nose\n\n\n\n\nimg = cv2.rectangle(img, (x0,y0), (x0+w0,y0+h0), (255,0,0), 2)\nimg = cv2.rectangle(img, (x1,y1), (x1+w1,y1+h1), (255,0,0), 2)\nimg = cv2.rectangle(img, (x2,y2), (x2+w2,y2+h2),

In [3]:
# Apply LBP Algorithm on a block of the image
def get_lbp_hist(grayscale_img):
    
    dim = grayscale_img.shape
    
    lbp = np.zeros(256)
    
    num1=num2=0
    
    
    #print(grayscale_img.shape)
    for i in range(1,dim[0]-1):
        for j in range(1,dim[1]-1):
            
            num1+=1
            binary = ""
            if grayscale_img[i][j] < grayscale_img[i-1][j-1]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i-1][j]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i-1][j+1]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i][j+1]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i+1][j+1]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i+1][j]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i+1][j-1]:
                binary+='1'
            else:
                binary+='0'
                
            if grayscale_img[i][j] < grayscale_img[i][j-1]:
                binary+='1'
            else:
                binary+='0'
                
            
            transitions = 0
            
            for k in range(1,len(binary)):
                if binary[k] != binary[k-1]:
                    transitions +=1
            
            if transitions  > 2: 
                continue
                
            num2+=1
            
            binary = int(binary,2)
            lbp[binary]+=1
    
    return lbp # <LBP histogram which is a list of 256 numbers>




In [19]:
# Segment the image into 7x7 blocks,apply LBP algorithm on them, then Concatente all hitograms into one
def segment_img(img):
    dim = img.shape
    patch_width  = 7
    patch_height = 7
    
    
    histograms = []
    num = 0
    
    #print(img.shape)
    for x in range(0,dim[0],patch_width):
        for y in range(0,dim[1],patch_height):
            #print(1)
            
            if(patch_width+x >= dim[0] or patch_width+y >= dim[1] ):
                continue
            num+=1 
            patch = img[x:x+patch_width,y:y+patch_height]
    
            histogram = get_lbp_hist(patch)
            histograms.append(histogram)
    #print(1)
    histo = np.zeros(256 * len(histograms) )
    
    count = 0 
    #print(1)
    for h in histograms:
        for i in range( len(h) ):
            histo[count] = h[i]
            count +=1
    
    #print(len(histo))
    #for h in histograms:
    #    for i in range(len(h)):
    #        histo[i] += h[i]
        
        
    return histo

In [20]:
# Training on images for "tom" and "clinton"

classes = []
train_hist  = []
train_labels = []

def train_data():
    
    
    for person in os.listdir('training'):
        
        classes.append(person)
        
        for image in os.listdir('training/'+person):
            
            img_path = 'training/'+person+'/'+image
            img = cv2.imread(img_path)
            img,_ = face_detection(img)
        
            histo = segment_img(img)
            train_hist.append(histo)
            train_labels.append(person)
train_data()

In [22]:
print(train_labels)
print ( len(train_hist)  )

['ezzat', 'ezzat', 'ezzat', 'ezzat', 'ezzat', 'clinton', 'clinton', 'clinton', 'clinton', 'clinton', 'tom', 'tom', 'tom', 'tom', 'tom']
15


In [35]:
def KNN(all_hist,test_hist,k):
    pass


# Apply Nearest Neighbour Algorithm for test image 
def classify(img):
    
    img,_ = face_detection(img)
    
    test_hist = segment_img(img)
    
    
    mini_dist  = 1000000
    mini_class = -1
    
    
    #print(len(train_hist[0]))
    
    #print(len(test_hist))
    for index in range(len(train_hist)) :
        d = 0 
        
        for i in range(len(test_hist)):
            d += abs(train_hist[index][i] - test_hist[i])
        
        if d < mini_dist :
            mini_dist = d 
            mini_class = train_labels[index]
            
            
    print(mini_dist)
    
    #if mini_dist > 5500:
    #    return -1
    
    return mini_class

In [9]:
def test_img(img):
    #cv2.imshow('Test Image',img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    c = classify(img)    
    if c != -1:
        print (c)
    else:
        print("No Match")
        c= "No Match"
        
    return c
    

In [40]:
cap = cv2.VideoCapture(0) 

while 1: 
    ret, img = cap.read() 
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
    
    
    res = face_detection(img)
    #print(res)
    
    if res != -1:
        img2 = res[0]
        face = res[1]
        (x,y,w,h) = face
        
        c = test_img(img)
        
        img = cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
        img = cv2.putText(img, c, (x,y), cv2.FONT_HERSHEY_PLAIN, 2.5, (0, 0, 255), 2)
    
    cv2.imshow('img',img) 
    
    k = cv2.waitKey(30) & 0xff
    if k == 27: 
        break
cap.release() 
cv2.destroyAllWindows() 

5976.0
clinton
5946.0
clinton
5952.0
ezzat
5945.0
clinton
5990.0
clinton
5928.0
ezzat
5837.0
ezzat
5771.0
ezzat
5713.0
ezzat
5841.0
ezzat
5863.0
ezzat
5786.0
ezzat
5949.0
ezzat
5740.0
ezzat
5845.0
ezzat
5889.0
ezzat
5994.0
ezzat
5882.0
ezzat
5893.0
ezzat
5892.0
ezzat
5871.0
ezzat
5859.0
ezzat
5912.0
ezzat
5891.0
ezzat
5931.0
ezzat


In [54]:
img = cv2.imread('test/16.jpg')
test_img(img)

4976.0
ezzat


'ezzat'